In [66]:
import os, tarfile
import re, glob
import pandas as pd
import numpy as np
import zipfile, gzip
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import itertools
from Bio import SeqIO

In [50]:
vcf_file_path = "/data/private/pdutta/Collab_data/Oliver_data/APOE_TREM2/TREM2_new.vcf"
reference_file_path = "/data/projects/Resources/HumanReferenceGenome/chr19.fa"
output_path = "/data/private/pdutta/Collab_data/Oliver_data/DNABERT_data/TREM2_CrProm"

In [51]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

## Getting the VCF file and get the coordinates and alternative allele information 

In [52]:
vcf_file = pysam.VariantFile(vcf_file_path)
reference_fasta = pysam.FastaFile(reference_file_path)

In [77]:
position_list = []
alternative_allele_list = []
for rec in vcf_file.fetch():
    # You can access all data about the variant like this:
    print('Alternative Allele',  rec.alts, rec.chrom, rec.pos, rec.ref)
    position_list.append(rec.pos)
    alternative_allele_list.append(rec.alts[0])

Alternative Allele ('A',) 6 41158608 C
Alternative Allele ('T',) 6 41158620 C
Alternative Allele ('A',) 6 41158657 C
Alternative Allele ('A',) 6 41158680 G
Alternative Allele ('T',) 6 41158685 G
Alternative Allele ('T',) 6 41158691 C
Alternative Allele ('T',) 6 41158720 C
Alternative Allele ('G',) 6 41158721 C
Alternative Allele ('T',) 6 41158747 C
Alternative Allele ('T',) 6 41158767 C
Alternative Allele ('A',) 6 41158768 G
Alternative Allele ('A',) 6 41158875 G
Alternative Allele ('A',) 6 41158881 G
Alternative Allele ('C',) 6 41158904 A
Alternative Allele ('G',) 6 41158917 A
Alternative Allele ('C',) 6 41158927 G
Alternative Allele ('T',) 6 41158963 C
Alternative Allele ('A',) 6 41159042 G
Alternative Allele ('T',) 6 41159052 C
Alternative Allele ('C',) 6 41159063 G
Alternative Allele ('G',) 6 41159068 T
Alternative Allele ('A',) 6 41159805 G
Alternative Allele ('T',) 6 41159823 C
Alternative Allele ('A',) 6 41159845 G
Alternative Allele ('T',) 6 41159867 C
Alternative Allele ('A',)

In [54]:
min_position = min(position_list)
max_position = max(position_list)
print(min_position, max_position)

41158608 41163134


In [55]:
position_list= np.array(position_list)
alternative_allele_list = np.array(alternative_allele_list)

## Get the co-ordinates from Core Promoter

In [56]:
core_prom_bed = pd.read_csv("/data/private/pdutta/DNABERT_data/Core_promoters/Core_Prom_bed.tsv", sep= "\t")
core_prom_bed

,Chr,gene_id,transcript_id,exon_id,strand,TSS,start,end
0,chr7,ENSG00000004059.11,ENST00000000233.10,ENSE00001872691.2,+,127588411,127588366,127588455
1,chr7,ENSG00000001626.16,ENST00000003084.11,ENSE00001343851.2,+,117480025,117479980,117480069
2,chr7,ENSG00000001630.17,ENST00000003100.13,ENSE00001216550.10,-,92134477,92134433,92134522
3,chr7,ENSG00000002933.9,ENST00000004103.8,ENSE00001827596.2,+,150800769,150800724,150800813
4,chr7,ENSG00000004799.8,ENST00000005178.6,ENSE00001133344.6,-,95596516,95596472,95596561
...,...,...,...,...,...,...,...,...
236970,chr15,ENSG00000103876.14,ENST00000684569.1,ENSE00003921718.1,+,80153010,80152965,80153054
236971,chr15,ENSG00000213614.11,ENST00000684602.1,ENSE00002620879.1,-,72375989,72375945,72376034
236972,chr15,ENSG00000213614.11,ENST00000684667.1,ENSE00002584031.1,-,72375972,72375928,72376017
236973,chr15,ENSG00000104133.16,ENST00000684676.1,ENSE00003922470.1,-,44663688,44663644,44663733


In [57]:
core_prom_19  = core_prom_bed[core_prom_bed['Chr']=='chr6'].reset_index(drop=True)
core_prom_19

,Chr,gene_id,transcript_id,exon_id,strand,TSS,start,end
0,chr6,ENSG00000001036.14,ENST00000002165.11,ENSE00003705756.2,-,143511720,143511676,143511765
1,chr6,ENSG00000008197.5,ENST00000008391.4,ENSE00001843453.2,+,50713526,50713481,50713570
2,chr6,ENSG00000010017.13,ENST00000011619.6,ENSE00001815244.2,-,13711835,13711791,13711880
3,chr6,ENSG00000010818.10,ENST00000012134.7,ENSE00003804252.1,-,142945031,142944987,142945076
4,chr6,ENSG00000026297.17,ENST00000028008.9,ENSE00001855489.2,-,166956584,166956540,166956629
...,...,...,...,...,...,...,...,...
10995,chr6,ENSG00000155906.20,ENST00000684715.1,ENSE00003917755.1,-,151452146,151452102,151452191
10996,chr6,ENSG00000135317.14,ENST00000684717.1,ENSE00003916063.1,-,85593897,85593853,85593942
10997,chr6,ENSG00000172469.17,ENST00000684753.1,ENSE00003920710.1,+,95577865,95577820,95577909
10998,chr6,ENSG00000155906.20,ENST00000684765.1,ENSE00003917755.1,-,151452146,151452102,151452191


In [58]:
core_prom_19[(core_prom_19['start']>=min_position)]

,Chr,gene_id,transcript_id,exon_id,strand,TSS,start,end
0,chr6,ENSG00000001036.14,ENST00000002165.11,ENSE00003705756.2,-,143511720,143511676,143511765
1,chr6,ENSG00000008197.5,ENST00000008391.4,ENSE00001843453.2,+,50713526,50713481,50713570
3,chr6,ENSG00000010818.10,ENST00000012134.7,ENSE00003804252.1,-,142945031,142944987,142945076
4,chr6,ENSG00000026297.17,ENST00000028008.9,ENSE00001855489.2,-,166956584,166956540,166956629
5,chr6,ENSG00000047932.14,ENST00000052569.10,ENSE00000919104.8,-,117602528,117602484,117602573
...,...,...,...,...,...,...,...,...
10995,chr6,ENSG00000155906.20,ENST00000684715.1,ENSE00003917755.1,-,151452146,151452102,151452191
10996,chr6,ENSG00000135317.14,ENST00000684717.1,ENSE00003916063.1,-,85593897,85593853,85593942
10997,chr6,ENSG00000172469.17,ENST00000684753.1,ENSE00003920710.1,+,95577865,95577820,95577909
10998,chr6,ENSG00000155906.20,ENST00000684765.1,ENSE00003917755.1,-,151452146,151452102,151452191


## Get the Core promoter regions which contains the variant information

In [59]:
position_list

array([41158608, 41158620, 41158657, 41158680, 41158685, 41158691,
       41158720, 41158721, 41158747, 41158767, 41158768, 41158875,
       41158881, 41158904, 41158917, 41158927, 41158963, 41159042,
       41159052, 41159063, 41159068, 41159805, 41159823, 41159845,
       41159867, 41159868, 41159875, 41159877, 41159882, 41161366,
       41161395, 41161450, 41161457, 41161469, 41161470, 41161514,
       41161515, 41161519, 41161537, 41161557, 41161562, 41161592,
       41161606, 41161607, 41161608, 41163086, 41163134])

In [60]:
min_position

41158608

In [62]:
core_prom_19['vcf_position'] = pd.Series([[] for _ in range(len(core_prom_19))], index=core_prom_19.index)
core_prom_19['alts'] = pd.Series([[] for _ in range(len(core_prom_19))], index=core_prom_19.index)

for alt, position in zip(alternative_allele_list, position_list):
    # Check if the position falls within the range of any row
    mask = (core_prom_19['start'] <= position) & (core_prom_19['end'] >= position)
    # If it does, store the index of the row
    #core_prom_19.loc[mask, 'vcf_position'] = core_prom_19.loc[mask, 'vcf_position'].apply(lambda x: x.append(position) if pd.notnull(x) else [position])
    core_prom_19.loc[mask, 'vcf_position'] = core_prom_19.loc[mask, 'vcf_position'].apply(lambda x: x + [position])
    core_prom_19.loc[mask, 'alts'] = core_prom_19.loc[mask, 'alts'].apply(lambda x: x + [alt])

In [63]:
selected_rows= core_prom_19[core_prom_19['vcf_position'].apply(lambda x: len(x) > 0)]
selected_rows

,Chr,gene_id,transcript_id,exon_id,strand,TSS,start,end,vcf_position,alts
1429,chr6,ENSG00000095970.17,ENST00000373113.8,ENSE00001922142.2,-,41163116,41163072,41163161,"[41163086, 41163134]","[G, T]"
1430,chr6,ENSG00000095970.17,ENST00000373122.8,ENSE00002318238.2,-,41163103,41163059,41163148,"[41163086, 41163134]","[G, T]"


In [64]:
print(min_position , max_position)

41158608 41163134


In [67]:
genome = SeqIO.to_dict(SeqIO.parse("/data/projects/Resources/Gencode_genome_annotation/GRCh38.primary_assembly.genome.fa", "fasta"))

In [90]:
data = []
for _, row in selected_rows.iterrows():
    chrom = row['Chr']
    ref_start = row['start']
    ref_end = row['end']
    variant_position_list = row['vcf_position']
    alts_list = row['alts']

    # Get reference sequence
    ref_seq = str(genome[chrom].seq[ref_start:ref_end])
    print(ref_seq)
    
    for variant_position, alt in zip(variant_position_list, alts_list):
        seq_variant_pos = variant_position - ref_start
        print(ref_start, variant_position, seq_variant_pos, alt)
        alt_seq = ref_seq[:seq_variant_pos-1] + alt + ref_seq[seq_variant_pos:]
        print(alt_seq, ref_seq[seq_variant_pos-1])
        data.append({
                'chr': chrom,
                'strand': row['strand'],
                'Core_Prom_start': ref_start,
                'Core_Prom_end': ref_end,
                'VCF_position': variant_position,
                'ref_neucleotide': ref_seq[seq_variant_pos-1],
                'alt_neucleotide': alt,
                'reference_seq': ref_seq,
                'alt_seq': alt_seq
            })
new_df = pd.DataFrame(data)

GAGGCTCCATGCCACCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGCCTTGTGCAAGATCTCGTCTTTCCCTTGAACTGCAGAAAAGAGA
41163072 41163086 14 G
GAGGCTCCATGCCGCCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGCCTTGTGCAAGATCTCGTCTTTCCCTTGAACTGCAGAAAAGAGA A
41163072 41163134 62 T
GAGGCTCCATGCCACCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGCCTTGTGCAAGATCTCTTCTTTCCCTTGAACTGCAGAAAAGAGA G
ATGAGCAGCCGGAGAGGCTCCATGCCACCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGCCTTGTGCAAGATCTCGTCTTTCCCTTGAAC
41163059 41163086 27 G
ATGAGCAGCCGGAGAGGCTCCATGCCGCCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGCCTTGTGCAAGATCTCGTCTTTCCCTTGAAC A
41163059 41163134 75 T
ATGAGCAGCCGGAGAGGCTCCATGCCACCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGCCTTGTGCAAGATCTCTTCTTTCCCTTGAAC G


In [91]:
new_df

,chr,strand,Core_Prom_start,Core_Prom_end,VCF_position,ref_neucleotide,alt_neucleotide,reference_seq,alt_seq
0,chr6,-,41163072,41163161,41163086,A,G,GAGGCTCCATGCCACCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGC...,GAGGCTCCATGCCGCCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGC...
1,chr6,-,41163072,41163161,41163134,G,T,GAGGCTCCATGCCACCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGC...,GAGGCTCCATGCCACCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGC...
2,chr6,-,41163059,41163148,41163086,A,G,ATGAGCAGCCGGAGAGGCTCCATGCCACCCTTCCCCAGCCAAGGGC...,ATGAGCAGCCGGAGAGGCTCCATGCCGCCCTTCCCCAGCCAAGGGC...
3,chr6,-,41163059,41163148,41163134,G,T,ATGAGCAGCCGGAGAGGCTCCATGCCACCCTTCCCCAGCCAAGGGC...,ATGAGCAGCCGGAGAGGCTCCATGCCACCCTTCCCCAGCCAAGGGC...


In [95]:
# Melt the DataFrame to a long format
df_data = pd.DataFrame(columns=['chr', 'strand','Core_Prom_start', 'Core_Prom_end', "VCF_position", "Data" ,"Label"])
for i in range(len(new_df)):
    df_data = df_data.append({'chr': new_df.iloc[i]['chr'], 'strand': new_df.iloc[i]['strand'], 'Core_Prom_start': new_df.iloc[i]['Core_Prom_start'], 'Core_Prom_end': new_df.iloc[i]['Core_Prom_end'], 'VCF_position': new_df.iloc[i]['VCF_position'], "Data": new_df.iloc[i]['reference_seq'], 'Label': 1}, ignore_index=True)
    df_data = df_data.append({'chr': new_df.iloc[i]['chr'], 'strand': new_df.iloc[i]['strand'], 'Core_Prom_start': new_df.iloc[i]['Core_Prom_start'], 'Core_Prom_end': new_df.iloc[i]['Core_Prom_end'], 'VCF_position': new_df.iloc[i]['VCF_position'], "Data": new_df.iloc[i]['alt_seq'], 'Label': 0}, ignore_index=True)

/tmp/ipykernel_1993411/2717605502.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_data = df_data.append({'chr': new_df.iloc[i]['chr'], 'strand': new_df.iloc[i]['strand'], 'Core_Prom_start': new_df.iloc[i]['Core_Prom_start'], 'Core_Prom_end': new_df.iloc[i]['Core_Prom_end'], 'VCF_position': new_df.iloc[i]['VCF_position'], "Data": new_df.iloc[i]['reference_seq'], 'Label': 1}, ignore_index=True)
/tmp/ipykernel_1993411/2717605502.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_data = df_data.append({'chr': new_df.iloc[i]['chr'], 'strand': new_df.iloc[i]['strand'], 'Core_Prom_start': new_df.iloc[i]['Core_Prom_start'], 'Core_Prom_end': new_df.iloc[i]['Core_Prom_end'], 'VCF_position': new_df.iloc[i]['VCF_position'], "Data": new_df.iloc[i]['alt_seq'], 'Label': 0}, ignore_index=True)
/tmp/ipykernel_1993411

In [96]:
df_data

,chr,strand,Core_Prom_start,Core_Prom_end,VCF_position,Data,Label
0,chr6,-,41163072,41163161,41163086,GAGGCTCCATGCCACCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGC...,1
1,chr6,-,41163072,41163161,41163086,GAGGCTCCATGCCGCCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGC...,0
2,chr6,-,41163072,41163161,41163134,GAGGCTCCATGCCACCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGC...,1
3,chr6,-,41163072,41163161,41163134,GAGGCTCCATGCCACCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGC...,0
4,chr6,-,41163059,41163148,41163086,ATGAGCAGCCGGAGAGGCTCCATGCCACCCTTCCCCAGCCAAGGGC...,1
5,chr6,-,41163059,41163148,41163086,ATGAGCAGCCGGAGAGGCTCCATGCCGCCCTTCCCCAGCCAAGGGC...,0
6,chr6,-,41163059,41163148,41163134,ATGAGCAGCCGGAGAGGCTCCATGCCACCCTTCCCCAGCCAAGGGC...,1
7,chr6,-,41163059,41163148,41163134,ATGAGCAGCCGGAGAGGCTCCATGCCACCCTTCCCCAGCCAAGGGC...,0


In [97]:
def seq2kmer(seq, k):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space
    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [98]:
df_data['Sequence'] = df_data['Data'].apply(seq2kmer, args=(6,))
df_data

,chr,strand,Core_Prom_start,Core_Prom_end,VCF_position,Data,Label,Sequence
0,chr6,-,41163072,41163161,41163086,GAGGCTCCATGCCACCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGC...,1,GAGGCT AGGCTC GGCTCC GCTCCA CTCCAT TCCATG CCAT...
1,chr6,-,41163072,41163161,41163086,GAGGCTCCATGCCGCCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGC...,0,GAGGCT AGGCTC GGCTCC GCTCCA CTCCAT TCCATG CCAT...
2,chr6,-,41163072,41163161,41163134,GAGGCTCCATGCCACCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGC...,1,GAGGCT AGGCTC GGCTCC GCTCCA CTCCAT TCCATG CCAT...
3,chr6,-,41163072,41163161,41163134,GAGGCTCCATGCCACCCTTCCCCAGCCAAGGGCAGAAGCAGAGTGC...,0,GAGGCT AGGCTC GGCTCC GCTCCA CTCCAT TCCATG CCAT...
4,chr6,-,41163059,41163148,41163086,ATGAGCAGCCGGAGAGGCTCCATGCCACCCTTCCCCAGCCAAGGGC...,1,ATGAGC TGAGCA GAGCAG AGCAGC GCAGCC CAGCCG AGCC...
5,chr6,-,41163059,41163148,41163086,ATGAGCAGCCGGAGAGGCTCCATGCCGCCCTTCCCCAGCCAAGGGC...,0,ATGAGC TGAGCA GAGCAG AGCAGC GCAGCC CAGCCG AGCC...
6,chr6,-,41163059,41163148,41163134,ATGAGCAGCCGGAGAGGCTCCATGCCACCCTTCCCCAGCCAAGGGC...,1,ATGAGC TGAGCA GAGCAG AGCAGC GCAGCC CAGCCG AGCC...
7,chr6,-,41163059,41163148,41163134,ATGAGCAGCCGGAGAGGCTCCATGCCACCCTTCCCCAGCCAAGGGC...,0,ATGAGC TGAGCA GAGCAG AGCAGC GCAGCC CAGCCG AGCC...


In [99]:
#df_selected.to_csv(output_path+"/selected_Donor_splice_sites.tsv", sep="\t", index=False)
new_df.to_csv(output_path+"/intermediate.tsv",  sep="\t", index=False)
df_data[['Sequence','Label']].to_csv(output_path+"/dev.tsv" , sep="\t", index=False)